In [119]:
import pandas as pd
import json
import numpy as np

## Books

In [84]:
book_info_path = "backend/resources/NLB_updated.csv"
recommendation_path = "backend/resources/recommendation.csv"

In [163]:
columns = ["identifier_uuid",
    "nlb_type",
    "language",
    "abstract",
    "date_created",
    "merged_creator",
    "title"]

In [164]:
book_info = pd.read_csv(book_info_path, usecols=columns) \
    .rename(columns={"identifier_uuid": "uuid", "nlb_type": "type"}) \
    .fillna("unknown")

In [170]:
def to_books_dynamo_csv(book_info, size=10):
    keys = []
    data = []
    for i, chunk in enumerate(np.array_split(book_info, size)):
        keys.append(str(i))
        data.append(json.dumps(chunk.drop(columns="abstract").to_dict("records"), sort_keys=True, separators=(',', ':')))
    print(size)
    return pd.DataFrame({
        "key": keys,
        "data": data
    })

In [171]:
to_books_dynamo_csv(book_info) \
    .to_csv("db/resources/books.csv", index=False)

10


/opt/miniconda3/envs/cs5224-project/lib/python3.9/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [167]:
book_info.to_csv("db/resources/book_info.csv", index=False)

In [131]:
recommendations = pd.read_csv(recommendation_path)

In [107]:
def to_recommendations_dynamo_csv(recommendations):
    uuid = recommendations["UUID"]
    rec_list = [json.dumps(rec.tolist()) for rec in recommendations.iloc[:, 1:].values]
    return pd.DataFrame({
        "uuid": uuid,
        "data": rec_list
    })

In [108]:
to_recommendations_dynamo_csv(recommendations) \
    .to_csv("db/resources/recommendations.csv", index=False)
